# Simple Email
This Jupyter notebook runs on Colab and demonstrates composing a simple email.

## Install Ollama

Before we get started with Mellea, we download, install and serve ollama. We define set_css to wrap Colab output.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &

from IPython.display import HTML, display


def set_css():
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))


get_ipython().events.register("pre_run_cell", set_css)

## Install Mellea
We run `uv pip install mellea` to install Mellea.

In [ ]:
!uv pip install mellea -q

## Import Mellea and Start a Session
We initialize a backend running Ollama using the granite3.3-chat model.

In [ ]:
import mellea

m = mellea.start_session()

## Basic Email
We make a simple email request.

In [ ]:
email = m.instruct("Write an email inviting interns to an office party at 3:30pm.")
print(str(email))

## Email Function with User Variables
We wrap this call into a function with some arguments.

In [ ]:
def write_email(m: mellea.MelleaSession, name: str, notes: str) -> str:
    email = m.instruct(
        "Write an email to {{name}} using the notes following: {{notes}}.",
        user_variables={"name": name, "notes": notes},
    )
    return email.value  # str(email) also works.


print(
    write_email(
        m,
        "Olivia",
        "Olivia helped the lab over the last few weeks by organizing intern events, advertising the speaker series, and handling issues with snack delivery.",
    )
)

## Email with Requirements
Suppose we want to ensure that the email has a salutation and contains only lower-case letters.
We capture these post-conditions by specifying requirements on the m.instruct call.

In [ ]:
def write_email_with_requirements(
    m: mellea.MelleaSession, name: str, notes: str
) -> str:
    email = m.instruct(
        "Write an email to {{name}} using the notes following: {{notes}}.",
        requirements=[
            "The email should have a salutation",
            "Use only lower-case letters",
        ],
        user_variables={"name": name, "notes": notes},
    )
    return str(email)


print(
    write_email_with_requirements(
        m,
        name="Olivia",
        notes="Olivia helped the lab over the last few weeks by organizing intern events, advertising the speaker series, and handling issues with snack delivery.",
    )
)

## Email with Rejection Sampling
We add two requirements to the instruction which will be added to the model request. 
But we don't check yet if these requirements are satisfied, we add a strategy for validating the requirements.

This sampling strategy (RejectionSamplingStrategy()) checks if all requirements are met and if any requirement fails, the sampling strategy will sample a new email from the LLM.

In [ ]:
from mellea.stdlib.sampling import RejectionSamplingStrategy


def write_email_with_strategy(m: mellea.MelleaSession, name: str, notes: str) -> str:
    email_candidate = m.instruct(
        "Write an email to {{name}} using the notes following: {{notes}}.",
        requirements=[
            "The email should have a salutation",
            "Use only lower-case letters",
        ],
        strategy=RejectionSamplingStrategy(loop_budget=5),
        user_variables={"name": name, "notes": notes},
        return_sampling_results=True,
    )
    if email_candidate.success:
        return str(email_candidate.result)
    else:
        print("Expect sub-par result.")
        return email_candidate.sample_generations[0].value


print(
    write_email_with_strategy(
        m,
        "Olivia",
        "Olivia helped the lab over the last few weeks by organizing intern events, advertising the speaker series, and handling issues with snack delivery.",
    )
)